In [ ]:
####################### I M P O R T S ###############################

import os
import glob
import numpy as np
from os.path import dirname, basename
from openmm.app import *
from openmm import *
from openmm.unit import kelvin, nanometer, picoseconds
from sys import stdout

#####################################################################

# Minimize Structure #

If there are problems, run 'openmm-setup'

In [ ]:
####################### C O N F I G #################################

PDB_FOLDER = "backmapped/PED/atomistic/test/"
PDB_REGEX = "**/backmapped_fixed_*.pdb"
TOLERANCE = 500 # kilojoules_per_mole/nanometer
FOLDER_NAME = "minimized"
ADD_MEMBRANE = False
SOLVATE = False
RESTRAIN_ATOMS = ['CA'] # ['N', 'CA', 'C', 'O']

#####################################################################

def minimize(pdb_in_filename, pdb_out_filename):
        pdb = PDBFile(pdb_in_filename)
        forcefield = ForceField('amber14-all.xml')
        modeller = Modeller(pdb.topology, pdb.positions)

        system = forcefield.createSystem(
                modeller.topology,
                nonbondedMethod=PME,
                nonbondedCutoff=1*nanometer,
                constraints=HBonds
        )
        system.setDefaultPeriodicBoxVectors(Vec3(10., 0., 0.), Vec3(0., 10., 0.), Vec3(0., 0., 10.))

        # --- RESTRAINTS --- #

        # restraint = CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2')
        # restraint.addGlobalParameter('k', 1000.*unit.kilojoules_per_mole/unit.nanometers**2)
        # restraint.addPerParticleParameter('x0')
        # restraint.addPerParticleParameter('y0')
        # restraint.addPerParticleParameter('z0')

        # for atom in modeller.topology.atoms():
        #         if atom.name in RESTRAIN_ATOMS:
        #                 restraint.addParticle(atom.index, modeller.positions[atom.index])
        
        # system.addForce(restraint)

        ######################

        integrator = LangevinMiddleIntegrator(300*kelvin, 1/picoseconds, 0.001*picoseconds)
        platform = Platform.getPlatformByName('CUDA')
        prop = dict(CudaPrecision='single')
        simulation = Simulation(modeller.topology, system, integrator, platform, prop)

        print('Running Minimization...')
        
        simulation.context.setPositions(modeller.positions)
        simulation.minimizeEnergy(tolerance=TOLERANCE*unit.kilojoules_per_mole/unit.nanometer, maxIterations=1000)
        state = simulation.context.getState(getPositions=True, getEnergy=True, getForces=True)
        f = np.array([[a.x,a.y,a.z] for a in state.getForces()])
        e = state.getPotentialEnergy()
        print(f'force: {np.linalg.norm(f, axis=1).mean()} | energy: {e._value} | fmax: {f.max()} | arg: {np.argmax(f.max(axis=1))}')
        if np.linalg.norm(f, axis=1).mean() > 2 * TOLERANCE:
                print(f"Failed minimization for {pdb_in_filename}")
                return

        simulation.step(50000)
        print('Saving...')
        positions = simulation.context.getState(getPositions=True).getPositions()
        PDBFile.writeFile(simulation.topology, positions, open(pdb_out_filename, 'w'))
        print('Done')

for pdb_in_filename in [log for log in glob.glob(os.path.join(PDB_FOLDER, PDB_REGEX), recursive=True) if not basename(dirname(log)) == 'minimized']:
        pdb_folder = dirname(pdb_in_filename)
        minimized_pdb_folder = os.path.join(pdb_folder, FOLDER_NAME)
        os.makedirs(minimized_pdb_folder, exist_ok=True)
        pdb_out_filename = os.path.join(
                minimized_pdb_folder,
                basename(pdb_in_filename),
        )
        if os.path.isfile(pdb_out_filename):
                continue
        minimize(pdb_in_filename, pdb_out_filename)
                

# Run Simulation #

In [ ]:
for filename in glob.glob(os.path.join(PDB_FOLDER, f"backmapped_*.pdb")):
        pdb = PDBFile(pdb_in_filename)
        forcefield = ForceField('amber14-all.xml')
        system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
                nonbondedCutoff=1*nanometer, constraints=HBonds)
        while system.getNumForces() > 0:
                system.removeForce(0)
        while system.getNumConstraints() > 0:
                system.removeConstraint(0)
        
        assert system.getNumConstraints() == 0
        assert system.getNumForces() == 0
        integrator = VerletIntegrator(0.001*picoseconds)
        simulation = Simulation(pdb.topology, system, integrator)
        simulation.context.setPositions(pdb.positions)

        simulation_pdb_folder = os.path.join(PDB_FOLDER, 'simulation')
        os.makedirs(simulation_pdb_folder, exist_ok=True)

        simulation.reporters.append(PDBReporter(os.path.join(simulation_pdb_folder, basename(filename)), 10000))
        simulation.reporters.append(
        StateDataReporter(
                stdout, 10000, step=True,potentialEnergy=True, temperature=True)
        )

        os.makedirs(os.path.join(simulation_pdb_folder), exist_ok=True)

        simulation.minimizeEnergy(tolerance=TOLERANCE*unit.kilojoules_per_mole/nanometer, maxIterations=10000)
        simulation.step(500000)
        break